# Deep Learning with Tensorflow on Amazon SageMaker and S3



## Imports

In [1]:
import sagemaker

print(sagemaker.__version__)
sess = sagemaker.Session()
# role = sagemaker.get_execution_role()
import os
from tensorflow import keras
import numpy as np

1.60.2


## Train on the notebook on local machine

In [26]:
!python train.py --augment False --bands 1 2 3 4 5 6 7 8 9 10 11 12 16 17 18 --batch-size 20 --epochs 2 --flip_left_right False --flip_up_down False --learning-rate 0.01 --gpu-count 0 --training ./ --validation ./ --model_dir ./ --numclasses 10 --rot90 False

TensorFlow version 2.4.0
Keras version 2.4.0
Using TensorFlow backend.
wandb: WARNING Keras version 2.2.4 is not fully supported. Required keras >= 2.4.0
wandb: Currently logged in as: margauxmforsythe (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.21
wandb: Syncing run curious-sun-87
wandb: ⭐️ View project at https://wandb.ai/margauxmforsythe/project_canopy
wandb: 🚀 View run at https://wandb.ai/margauxmforsythe/project_canopy/runs/2lcr5bik
wandb: Run data is saved locally in /Users/margauxmforstyhe/Desktop/workspace/AWS/wandb/run-20210317_111648-2lcr5bik
wandb: Run `wandb offline` to turn off syncing.

lr: 0.01, batch_size: 20, augmentation_data: False aug_flip_left_right False, aug_flip_up_down False, aug_rot90: True
bands [1]
Input shape: (100, 100, 1)
training_dir ./labels_test_v1.csv
val_dir ./val_labels.csv
Dataloader initialization

In [8]:
from sagemaker.tensorflow import TensorFlow

training_file = "labels_test_v1_m.csv"
validation_file = "val_labels_m.csv"

tf_estimator = TensorFlow(entry_point='train.py', 
                          role="AmazonSageMaker-ExecutionRole-20210311T123135",
#                           role="arn:aws:iam::963659202518:role/service-role/AmazonSageMaker-ExecutionRole-20210306T191865",
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='2.4', 
                          base_job_name='pc-tf-custom-container-test-job',
                          output_path='s3://margaux-bucket-us-east-1',
                          py_version='py37',
                          script_mode=True,
                          hyperparameters={'training_file':training_file, 
                                           'validation_file': validation_file,
                                            'bucket':"margaux-bucket-us-east-1",
                                           'epochs': 2, 'augment': False, 'batch-size': 20,
                                          'learning-rate': 0.01, 'flip_left_right': False,
                                          'flip_up_down': False, 'rot90': False, 'numclasses': 10, 
                                           'bands': "1 2 3 4 5 6 7 8 9 10 11 12 16 17 18"}
                         )

Training

In [9]:
tf_estimator.fit({'training': f"file://{training_file}", 
                 'validation': f"file://{validation_file}"})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Creating tmpj9__6tf4_algo-1-b4crl_1 ... 
Attaching to tmpj9__6tf4_algo-1-b4crl_12mdone
algo-1-b4crl_1  | 2021-03-17 23:45:53.685104: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
algo-1-b4crl_1  | 2021-03-17 23:45:53.691993: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
algo-1-b4crl_1  | 2021-03-17 23:45:53.808932: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
algo-1-b4crl_1  | 2021-03-17 23:45:57,432 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-b4crl_1  | 2021-03-17 23:45:57,443 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-b4crl_1  | 2021-03-17 23:45:57,493 botocore.credentials INFO     Found credentials in environment variables.
algo-1-b

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


KeyboardInterrupt: 

## Train on Cloud Amazon SageMaker

In [6]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import Rule, rule_configs
from sagemaker import get_execution_role

training_file = "labels_test_v1.csv"
validation_file = "val_labels.csv"
tf_estimator = TensorFlow(entry_point='train.py', # mnist_keras_tf20_eager.py
                          role="arn:aws:iam::963659202518:role/service-role/AmazonSageMaker-ExecutionRole-20210306T191865",
#                           role=get_execution_role(),
                          train_instance_count=1, 
                          train_instance_type='ml.p3.16xlarge', # ml.p3.16xlarge, ml.p3.2xlarge, ml.g4dn.xlarge, ml.p3.8xlarge
                          framework_version='2.4', 
                          base_job_name='pc-tf-custom-container-test-job-margaux',
                          output_path='s3://canopy-production-ml-output',
                          py_version='py37',
                          script_mode=True,
                          train_use_spot_instances=True,        # Use spot instance
                          train_max_run=600,                    # Max training time
                          train_max_wait=3600,                  # Max training time + spot waiting time
                          hyperparameters={'training_file':training_file, 
                                           'validation_file': validation_file,
                                            'bucket':"canopy-production-ml",
                                           'epochs': 2, 'augment': False, 'batch-size': 20,
                                          'learning-rate': 0.01, 'flip_left_right': False,
                                          'flip_up_down': False, 'rot90': False, 'numclasses': 10, 
                                           'bands': "1 2 3 4 5 6 7 8 9 10 11 12 16 17 18"}
                         )

Training

In [7]:
# Train on the training and validation data sets stored in S3

tf_estimator.fit({'training': f"s3://canopy-production-ml/training_inputs/test_csv/{training_file}", 
                  'validation': f"s3://canopy-production-ml/training_inputs/test_csv/{validation_file}"})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-03-18 00:52:57 Starting - Starting the training job...
2021-03-18 00:52:59 Starting - Launching requested ML instances.........
2021-03-18 00:54:44 Starting - Preparing the instances for training......
2021-03-18 00:56:08 Downloading - Downloading input data
2021-03-18 00:56:08 Training - Downloading the training image...............
2021-03-18 00:58:31 Training - Training image download completed. Training in progress.2021-03-18 00:58:32.256015: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-03-18 00:58:32.260428: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-03-18 00:58:32.361961: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-18 00:58:32.467084: W tensorflow/core/profiler/internal/smprofiler_timelin